**Install libraries and Download pre-Trained model**


In [ ]:
#@title <h1>Step1: Setup Wav2Lip</h1>
#@markdown * Install dependency
#@markdown * Download pretrained model
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install ffmpeg-python

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

# Step 2
# Upload video
# Trim your video
# To Remove the audio

In [31]:
import os
!pip install yt-dlp
!rm -df youtube.mp4

#@markdown ___
from urllib import parse as urlparse
YOUTUBE_URL = 'https://www.youtube.com/watch?v=YMuuEv37s0o' #@param {type:"string"}
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]


# remove previous input video
!rm -f /content/sample_data/input_vid.mp4


#@markdown ___

#@markdown ### Trim the video (start, end) seconds
start = 00 #@param {type:"integer"}
end = 60 #@param {type:"integer"}
interval = end - start

#@markdown <font color="orange"> Note: ``the trimmed video must have face on all frames``

# Download the YouTube video using yt-dlp
!yt-dlp -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

# Cut the video using FFmpeg
!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 /content/sample_data/input_vid.mp4

# Preview the trimmed video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/sample_data/input_vid.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""<video width=600 controls><source src="{data_url}"></video>""")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 54.9 MB/s eta 0:00:00
[youtube] Extracting URL: https://www.youtube.com/watch?v=YMuuEv37s0o
[youtube] YMuuEv37s0o: Downloading webpage
[youtube] YMuuEv37s0o: Downloading ios player API JSON
[youtube] YMuuEv37s0o: Downloading android player API JSON
[youtube] YMuuEv37s0o: Downloading iframe API JS
[youtube] YMuuEv37s0o: Downloading player da7c2a60
[youtube] YMuuEv37s0o: Downloading web player API JSON
[youtube] YMuuEv37s0o: Downloading m3u8 information
[youtube] YMuuEv37s0o: Downloading initial data API JSON
[info] YMuuEv37s0o: Downloading 1 format(s): 628
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragm

#Step 3
# Resize your video

In [ ]:
import os
import shutil
from google.colab import drive
from google.colab import files
from IPython.display import HTML, clear_output
from base64 import b64encode
import moviepy.editor as mp


def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

def get_video_resolution(video_path):
    """Function to get the resolution of a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return (width, height)

def resize_video(video_path, new_resolution):
    """Function to resize a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    fourcc = int(video.get(cv2.CAP_PROP_FOURCC))
    fps = video.get(cv2.CAP_PROP_FPS)
    width, height = new_resolution
    output_path = os.path.splitext(video_path)[0] + '_720p.mp4'
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    while True:
        success, frame = video.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, new_resolution)
        writer.write(resized_frame)
    video.release()
    writer.release()

#@markdown ### Select an uploading method
upload_method = "Custom Path" #@param ["Upload", "Custom Path"]


# remove previous input video
if os.path.isfile('/content/sample_data/input_vid.mp4'):
    os.remove('/content/sample_data/input_vid.mp4')

if upload_method == "Upload":
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, '/content/sample_data/input_vid.mp4')
    PATH_TO_YOUR_VIDEO = '/content/sample_data/input_vid.mp4'

elif upload_method == 'Custom Path':
    if not 'drive' in globals():
        drive.mount('/content/drive')
    #@markdown ``Add the full path to your video on your Gdrive `` 👇
    PATH_TO_YOUR_VIDEO = '/content/input_1_min.mp4' #@param {type:"string"}
    if not os.path.isfile(PATH_TO_YOUR_VIDEO):
        print("ERROR: File not found!")
        raise SystemExit(0)

#@markdown <font color="orange">Notes:

#@markdown <font color="orange">. ``If your uploaded video is 1080p or higher resolution, this cell will resize it to 720p.``

#@markdown <font color="orange">. ``Do not upload videos longer than 60 seconds.``

#@markdown ___

video_duration = mp.VideoFileClip(PATH_TO_YOUR_VIDEO).duration
if video_duration > 60:
    print("WARNING: Video duration exceeds 60 seconds. Please upload a shorter video.")
    raise SystemExit(0)

video_resolution = get_video_resolution(PATH_TO_YOUR_VIDEO)
print(f"Video resolution: {video_resolution}")
if video_resolution[0] >= 1920 or video_resolution[1] >= 1080:
    print("Resizing video to 720p...")
    os.system(f"ffmpeg -i {PATH_TO_YOUR_VIDEO} -vf scale=1280:720 /content/sample_data/input_vid.mp4")
    PATH_TO_YOUR_VIDEO = "/content/sample_data/input_vid.mp4"
    print("Video resized to 720p")
else:
    print("No resizing needed")

if upload_method == "Upload":
  clear_output()
  print("Input Video")
  showVideo(PATH_TO_YOUR_VIDEO)
else:
    if os.path.isfile(PATH_TO_YOUR_VIDEO):
        shutil.copyfile(PATH_TO_YOUR_VIDEO, "/content/sample_data/input_vid.mp4")
        print("Input Video")
        showVideo(PATH_TO_YOUR_VIDEO)


# Step 4
* upload your audio file directly to your colab notebook
* The length of video file and audio file should be same.
* Make the changes in your inference.py file /content/Wav2Lip/inference.py
add the below code to the main function to process with the non-face detected frames.
*   Add this code from line no 265 in inference.py file                                                       
        for p, f, c in zip(pred, frames, coords):
            if len(c) == 0:  # Check if there are no face coordinates (no face detected)
                out.write(f)  # Write the original frame without lip-syncing
            else:
                if len(c) == 4:  # Check if there are four values in the coordinates
                    y1, y2, x1, x2 = c
                    p = cv2.resize(p.astype(np.uint8), (x2 - x1, y2 - y1))
                    f[y1:y2, x1:x2] = p
                else:
                    print("Invalid coordinates:", c)  # Print a message for debugging purposes

                out.write(f)  # Write the lip-synced frame to the output video

# Step 5
* Give the path of your audio and video files after preprocessing.
* Change the value of pad_top, pad_bottom, pad_left, Pad_right as per your input video.
* pad_top =  0
* pad_bottom =  4
* pad_left =  0
* pad_right =  50
* rescaleFactor =  1
* nosmooth = False
* Strat training your model


In [30]:
#@title STEP4: Start Crunching and Preview Output
#@markdown <b>Note: Only change these, if you have to</b>
 pad_top =  0#@param {type:"integer"}
pad_bottom =  4#@param {type:"integer"}
pad_left =  0#@param {type:"integer"}
pad_right =  50#@param {type:"integer"}
rescaleFactor =  1#@param {type:"integer"}
nosmooth = False#@param {type:"boolean"}

if nosmooth == False:
  !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_vid.mp4" --audio "/content/output10-[AudioTrimmer.com]_1.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
else:
  !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_vid.mp4" --audio "/content/output10-[AudioTrimmer.com]_1.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth

#Preview output video
# clear_output()
# print("Final Video Preview")
# print("Download this video from", '/content/Wav2Lip/results/result_voice.mp4')
# showVideo('/content/Wav2Lip/results/result_voice.mp4')


Using cuda for inference.
Reading video frames...
Number of frames available for inference: 3000
(80, 4801)
Length of mel chunks: 2993
  0% 0/24 [00:00<?, ?it/s]
  0% 0/188 [00:00<?, ?it/s]
  1% 1/188 [02:53<9:00:09, 173.31s/it]
  1% 2/188 [02:56<3:47:41, 73.45s/it] 
  2% 3/188 [02:59<2:07:12, 41.26s/it]
  2% 4/188 [03:02<1:20:07, 26.13s/it]
  3% 5/188 [03:05<54:02, 17.72s/it]  
  3% 6/188 [03:08<38:26, 12.68s/it]
  4% 7/188 [03:11<28:44,  9.53s/it]
  4% 8/188 [03:14<22:10,  7.39s/it]
  5% 9/188 [03:17<17:47,  5.96s/it]
  5% 10/188 [03:20<14:52,  5.01s/it]
  6% 11/188 [03:23<13:05,  4.44s/it]
  6% 12/188 [03:25<11:33,  3.94s/it]
  7% 13/188 [03:28<10:29,  3.59s/it]
  7% 14/188 [03:31<09:47,  3.38s/it]
  8% 15/188 [03:34<09:28,  3.29s/it]
  9% 16/188 [03:37<09:00,  3.14s/it]
  9% 17/188 [03:40<08:41,  3.05s/it]
 10% 18/188 [03:43<08:27,  2.98s/it]
 10% 19/188 [03:46<08:21,  2.97s/it]
 11% 20/188 [03:49<08:35,  3.07s/it]
 11% 21/188 [03:52<08:36,  3.09s/it]
 12% 22/188 [03:55<08:20,  3.0

# Output Video

In [29]:
print("Final Video Preview")
print("Download this video from", '/content/Wav2Lip/results/result_voice.mp4')
showVideo('/content/Wav2Lip/results/result_voice.mp4')

Final Video Preview
Download this video from /content/Wav2Lip/results/result_voice.mp4
